# Rectangular data loader

## Closest ntile

In [ ]:
show_image(train_ds[3][0], figsize=(6,3))
x.shape

In [ ]:
asp_ratios = [operator.truediv(*Image.open(fn).size) for fn in train_ds.fns]
asp_ratios[:4]

In [ ]:
asp_ntiles = np.percentile(asp_ratios, [2,20,50,80,98])
asp_ntiles

In [ ]:
def closest_ntile(aspect, ntiles):
    return ntiles[np.argmin(abs(log(aspect)-log(ntiles)))]

In [ ]:
aspect = x.shape[2]/x.shape[1]
nearest_aspect = closest_ntile(aspect, asp_ntiles)
aspect,nearest_aspect

In [ ]:
target_px = 128*128; target_px

In [ ]:
target_r = int(math.sqrt(target_px/nearest_aspect))
target_c = int(target_r*nearest_aspect)
target_r,target_c,target_r*target_c

## SortAspectBatchSampler

In [ ]:
asp_nearests = [closest_ntile(o, asp_ntiles) for o in asp_ratios]
asp_nearests[:10]

In [ ]:
bs=32

In [ ]:
from itertools import groupby

In [ ]:
sort_nearest = sorted(enumerate(asp_nearests), key=itemgetter(1))
groups = [list(b) for a,b in groupby(sort_nearest, key=itemgetter(1))]
len(groups)

In [ ]:
groups[0][:5]

In [ ]:
sum(math.ceil(len(g)/bs) for g in groups)

In [ ]:
@dataclass
class SortAspectBatchSampler(Sampler):
    ds:Dataset; bs:int; shuffle:bool = False

    def __post_init__(self):
        asp_ratios = [operator.truediv(*Image.open(img).size) for img in self.ds.fns]
        asp_ntiles = np.percentile(asp_ratios, [2,20,50,80,98])
        asp_nearests = [closest_ntile(o, asp_ntiles) for o in asp_ratios]
        sort_nearest = sorted(enumerate(asp_nearests), key=itemgetter(1))
        self.groups = [list(b) for a,b in groupby(sort_nearest, key=itemgetter(1))]
        self.n = sum(math.ceil(len(g)/bs) for g in self.groups)

    def __len__(self): return self.n
    
    def __iter__(self):
        if self.shuffle: self.groups = sample(self.groups, len(self.groups))
        for group in self.groups:
            group = [(a,{'aspect':b}) for a,b in group]
            if self.shuffle: group = sample(group, len(group))
            for i in range(0, len(group), self.bs): yield group[i:i+self.bs]

In [ ]:
next(iter(SortAspectBatchSampler(train_ds, 4)))

In [ ]:
next(iter(SortAspectBatchSampler(train_ds, 4, True)))

## Rectangular dataset

In [ ]:
@dataclass
class TfmDataset(Dataset):
    ds: Dataset
    tfms: Collection[Callable]
        
    def __len__(self): return len(self.ds)
    
    def __getitem__(self,idx):
        if isinstance(idx, tuple): idx,kwargs = idx
        else: kwargs={}
        x,y = self.ds[idx]
        return apply_tfms(tfms)(x, **kwargs),y

In [ ]:
tfms = [
    rotate_tfm(degrees=(-20,20.), p=0.75),
    zoom_tfm(scale=(1.,1.5), p=1.)
]

In [ ]:
train_tds = TfmDataset(train_ds, tfms)

In [ ]:
_,axes = plt.subplots(2,2, figsize=(12,6))
for ax in axes.flat: show_image(train_tds[3][0], ax)